# Build The Positive Negative Interaction
Basically, all interactions that is recorded from Super Target will be labeled by 1. The rest is 0. The approach is like this
1. Create all combinations based on existing protein and compound
2. Combine Those DataFrame with known positive interactions
3. To get negative, simply remove the duplicate rows completely.
4. Combine the positive and negative back.
5. Save it

## Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_pos = pd.read_csv('../data/com-train-pos-final.csv')
train_pos.head()

,uniprot_id,drug,pubchemID,>=_4_H,>=_8_H,>=_16_H,>=_32_H,>=_1_Li,>=_2_Li,>=_1_B,...,SC1C(S)CCC1,SC1C(N)CCC1,SC1C(Cl)CCC1,SC1C(Br)CCC1,NC1C(N)CCC1,NC1C(Cl)CC1,NC1C(Br)CCC1,ClC1C(Cl)CCC1,ClC1C(Br)CCC1,BrC1C(Br)CCC1
0,HDAC3_HUMAN,"4-phenylimidazole, 17",24853793,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HDAC3_HUMAN,"4-phenylimidazole, 19",11700059,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HDAC3_HUMAN,AC1OCG0B,6918878,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HDAC3_HUMAN,Acetyldinaline,2746,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HDAC3_HUMAN,apicidin,467801,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Get the positive interactions

In [12]:
pos = train_pos[['uniprot_id', 'pubchemID']]
print(pos.shape)
pos.head()

(7902, 2)


,uniprot_id,pubchemID
0,HDAC3_HUMAN,24853793
1,HDAC3_HUMAN,11700059
2,HDAC3_HUMAN,6918878
3,HDAC3_HUMAN,2746
4,HDAC3_HUMAN,467801


## Create The Possible Interactions

In [5]:
uniprot_id = []
pubchem_id = []

for i in interactions.uniprot_id.unique():
    for j in interactions.pubchemID.unique():
        uniprot_id.append(i)
        pubchem_id.append(j)

print(len(uniprot_id))
print(len(pubchem_id))

131036
131036


In [10]:
# TOTAL INTERACTIONS
# PROTEIN * COMPOUND
# 17 * 7708

131036

In [36]:
all_ = pd.DataFrame()
all_['uniprot_id'] = uniprot_id
all_['pubchemID'] = pubchem_id
print(all_.shape)
all_.head()

(131036, 2)


,uniprot_id,pubchemID
0,HDAC3_HUMAN,24853793
1,HDAC3_HUMAN,11700059
2,HDAC3_HUMAN,6918878
3,HDAC3_HUMAN,2746
4,HDAC3_HUMAN,467801


## Combine The Dataset as One

In [37]:
7902 + 131036

138938

In [38]:
combined = pd.concat([pos, all_])
print(combined.shape)
combined.head()

(138938, 2)


,uniprot_id,pubchemID
0,HDAC3_HUMAN,24853793
1,HDAC3_HUMAN,11700059
2,HDAC3_HUMAN,6918878
3,HDAC3_HUMAN,2746
4,HDAC3_HUMAN,467801


## Get the negative interactions by removing duplicates

In [45]:
neg = combined.drop_duplicates(keep=False)
neg['interaction'] = 0
print(neg.shape)
neg.head()

(123134, 3)


<ipython-input-45-8ecdbb24f628>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg['interaction'] = 0


,uniprot_id,pubchemID,interaction
240,HDAC3_HUMAN,513439,0
241,HDAC3_HUMAN,513441,0
242,HDAC3_HUMAN,24193271,0
243,HDAC3_HUMAN,11016750,0
244,HDAC3_HUMAN,44333466,0


In [46]:
pos['interaction'] = 1
print(pos.shape)
pos.head()

(7902, 3)


,uniprot_id,pubchemID,interaction
0,HDAC3_HUMAN,24853793,1
1,HDAC3_HUMAN,11700059,1
2,HDAC3_HUMAN,6918878,1
3,HDAC3_HUMAN,2746,1
4,HDAC3_HUMAN,467801,1


In [41]:
# POS + NEG = TOTAL INTERACTION
7902 + 123134

131036

## Get the final train dataset

In [49]:
final = pd.concat([pos, neg])
print(final.shape)
final.head()

(131036, 3)


,uniprot_id,pubchemID,interaction
0,HDAC3_HUMAN,24853793,1
1,HDAC3_HUMAN,11700059,1
2,HDAC3_HUMAN,6918878,1
3,HDAC3_HUMAN,2746,1
4,HDAC3_HUMAN,467801,1


In [51]:
final.to_csv('../data/interactions.csv')

In [54]:
final['interaction'].value_counts()

0    123134
1      7902
Name: interaction, dtype: int64

In [53]:
final['interaction'].value_counts(normalize=True)

0    0.939696
1    0.060304
Name: interaction, dtype: float64

Because the data is imbalanced, we need to undersampled the negative interactions by proportion of 1:5